In [1]:
import pandas as pd
import plotly.express as px
from mainnet_launch.constants import (
    CACHE_TIME,
    eth_client,
    ALL_AUTOPOOLS,
    BAL_ETH,
    AUTO_ETH,
    AUTO_LRT,
    AutopoolConstants,
    WORKING_DATA_DIR,
)
from datetime import datetime, timedelta, timezone




In [24]:
from mainnet_launch.autopool_diagnostics.fees import fetch_autopool_fee_data

def compute_revenue_from_fees() -> dict[str,dict[str,float]]:
    """
    Fetch all the periodic and streaming fee events for each autopool then sum the ETH 
    value from the `fees` event by 7 day, 30 day and 1 year. then sum those value
    The data should look like
    
    {
        '7_days': {
            'autoETH_periodic': 1.671720333158365,
            'autoETH_streaming': 3.1315890919246083,
            'balETH_periodic': 0.7185848926255698,
            'balETH_streaming': 0.8942176501922965,
            'autoLRT_periodic': 0.4636031290053162,
            'autoLRT_streaming': 0.43119494960570615,
            'total_revenue': 7.310910046511863
        },
        '30_days': {
            'autoETH_periodic': 6.691773228438292,
            'autoETH_streaming': 11.565605559895612,
            'balETH_periodic': 3.039947675877742,
            'balETH_streaming': 4.815447767226244,
            'autoLRT_periodic': 1.9713912472088526,
            'autoLRT_streaming': 2.280668705081137,
            'total_revenue': 30.36483418372788
        },
        '1_year': {
            'autoETH_periodic': 8.596291557174354,
            'autoETH_streaming': 16.335460959034297,
            'balETH_periodic': 3.9371002057270115,
            'balETH_streaming': 7.222694296056844,
            'autoLRT_periodic': 2.5558998672512345,
            'autoLRT_streaming': 3.6563966618798514,
            'total_revenue': 42.30384354712359
        }
    }

    """
    fee_dfs = []
    for autopool in ALL_AUTOPOOLS:
        periodic_fee_df, streaming_fee_df = fetch_autopool_fee_data(autopool)
        periodic_fee_df.columns = [f'{autopool.name}_periodic']
        streaming_fee_df.columns = [f'{autopool.name}_streaming']
        fee_dfs.extend([periodic_fee_df, streaming_fee_df])

    fee_df = pd.concat(fee_dfs).fillna(0)
    
    today = datetime.now(timezone.utc)
    seven_days_ago = today - timedelta(days=7)
    thirty_days_ago = today - timedelta(days=30)
    year_ago = today - timedelta(days=365)

    revenue_in_last_7_days = fee_df[fee_df.index >= seven_days_ago].sum().to_dict()
    revenue_in_last_7_days['total_revenue'] = sum([revenue_in_last_7_days[k] for k in fee_df.columns])

    revenue_in_last_30_days = fee_df[fee_df.index >= thirty_days_ago].sum().to_dict()
    revenue_in_last_30_days['total_revenue'] = sum([revenue_in_last_30_days[k] for k in fee_df.columns])

    revenue_in_last_year = fee_df[fee_df.index >= year_ago].sum().to_dict()
    revenue_in_last_year['total_revenue'] = sum([revenue_in_last_year[k] for k in fee_df.columns])

    return {
        '7_days' : revenue_in_last_7_days,
        '30_days' :revenue_in_last_30_days, 
        '1_year': revenue_in_last_year
    }

revenue_data = compute_revenue_from_fees()
revenue_data

2024-10-29 09:38:01.172 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.173 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.174 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.176 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.176 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-29 09:38:01.177 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

{'7_days': {'autoETH_periodic': 1.671720333158365,
  'autoETH_streaming': 3.1315890919246083,
  'balETH_periodic': 0.7185848926255698,
  'balETH_streaming': 0.8942176501922965,
  'autoLRT_periodic': 0.4636031290053162,
  'autoLRT_streaming': 0.43119494960570615,
  'total_revenue': 7.310910046511863},
 '30_days': {'autoETH_periodic': 6.691773228438292,
  'autoETH_streaming': 11.565605559895612,
  'balETH_periodic': 3.039947675877742,
  'balETH_streaming': 4.815447767226244,
  'autoLRT_periodic': 1.9713912472088526,
  'autoLRT_streaming': 2.280668705081137,
  'total_revenue': 30.36483418372788},
 '1_year': {'autoETH_periodic': 8.596291557174354,
  'autoETH_streaming': 16.335460959034297,
  'balETH_periodic': 3.9371002057270115,
  'balETH_streaming': 7.222694296056844,
  'autoLRT_periodic': 2.5558998672512345,
  'autoLRT_streaming': 3.6563966618798514,
  'total_revenue': 42.30384354712359}}

In [ ]:
def compute_expenses_from_gas() -> dict[str,dict[str,float]]:
    """
    Get the ETH value spent on gas to run the onchain part of tokemak over the last 7, 30 and 365 days
    
    - Solver gas cost (as of Oct 29, the solver does not take any profit)
    - Keeper Network Gas Costs
    
    """

In [ ]:
keeper_df = pd.read_csv(WORKING_DATA_DIR / "chainlink_keeper_upkeeper_df.csv", index_col=0, parse_dates=True)[:-6]


def compute_protocol_level_profits():
    
    revenue = compute_revenue_from_fees()